# 前言

本项目根据以京东平台爬取的奥特曼、乐高、高达3类玩具的商品链接信息作为代表，对电商平台玩具的销售情况进行数据分析及可视化，试图解答以下问题：  
- 最受欢迎的产品是什么？  
- 什么品牌的产品单价最贵？  
- 单价高的产品是否存在溢价？  
- 能否给过年送礼一些建议？  
- 消费者更喜欢在旗舰店购买产品吗？

# 数据导入、组织与整理

## 准备

In [1]:
#导入所需的包
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import re
import seaborn as sns
import matplotlib.pyplot as plt
from imageio import imread
import numpy as np

## 导入数据

In [2]:
#导入数据
outman=pd.read_csv(r'/home/kesci/input/M787596/6k奥特曼.csv')
lego=pd.read_csv(r'/home/kesci/input/lego7292/京东-乐高.csv')
gundam=pd.read_csv(r'/home/kesci/input/Gundam1376/6K高达.csv')

## 数据处理

通过预览数据，发现在评价人数字段中，存在大量回车、制表符以及用汉字万代表万元的问题，需进行数据处理。  
另外，为了后续数据分析需要，需通过店铺名称判断，为每条数据进行分类（店铺名含“京东”的，认为是京东自营，含“旗舰店”的，认为是品牌旗舰店，其他店铺归为其他类。）

In [3]:
#定义函数，提取字符串中的数字，同时对字符串中包含“万“或无数字的情况进行处理。
def wan_remove(s):
    s=str(s)#将输入强制转换为字符串
    s=s.replace('\n','')
    s=s.replace('\t','')
    if re.match('.*万.*',s)!=None:#如果字符串中包含”万”，则提取字符串中的数字，并乘以10000作为结果返回
        r=float(re.match('(.*?)(\d+\.\d+|\d+)(万+)(.*)',s).group(2))*10000.0
    else:#如果不包含“万“
        try:#尝试提取数字，并作为结果返回
            r=float(re.match('(.*?)(\d+\.\d+|\d+)(.*)',s).group(2))
        except:#如果字符串为空或提取不到数字，则返回0
            r=0.0
    return r

In [4]:
#定义函数，为店铺分类，根据店铺属性，分为京东自营、品牌旗舰店和其他3类
def categorize(s):
    s=str(s)
    if re.match('.*京东.*',s)!=None:
        r='京东自营'
    elif re.match('.*旗舰.*',s)!=None:
        r='品牌旗舰店'
    else:
        r='其他'
    return r

In [5]:
#利用以上函数，从评价人数中抽取热度字段，从商家店名中抽取类型字段，按照商品SKU进行排重
for i in [outman, lego, gundam]:
    i['热度']=i.评价人数.apply(wan_remove)
    i['类型']=i.商家店名.apply(categorize)
    i.drop_duplicates(['商品SKU'], inplace=True)

In [6]:
#拼接整体数据toys，并按照商品SKU进行排重
toys=pd.DataFrame()
for i in [outman, lego, gundam]:
    toys=pd.concat([toys,i])
toys.drop_duplicates(['商品SKU'], inplace=True)
toys.reset_index(inplace=True,drop=True)

浏览数据：

In [7]:
toys.head()

,搜索关键词,商品名称,商品SKU,商品链接,封面图链接,价格,评价人数,评论链接,商家店名,店铺链接,标签,是否广告,页码,当前时间,页面网址,热度,类型
0,万代奥特曼,万代（BANDAI） 超可动 奥特曼 手办模型玩具 16cm 超可动 究极赛罗帕拉吉之盾,14018101327,https://item.jd.com/14018101327.html,https://img12.360buyimg.com/n7/jfs/t23992/257/...,99.0,\n\t\t\t\t\t\t\t7000+\n\t\t\t\t\t\t,https://item.jd.com/14018101327.html#comment,梅利玩具专营店,https://mall.jd.com/index-213319.html?from=pc,京东物流/\r\n2件9.8折/\r\n满赠/,否,1,2021-01-27 18:34:00,https://search.jd.com/Search?keyword=%E4%B8%87...,7000.0,其他
1,万代奥特曼,万代（BANDAI）罗布奥特曼变身器罗索布鲁水晶DX泽塔升华器勋章收纳盒回旋闪光儿童玩具 豪...,70265614548,https://item.jd.com/70265614548.html,https://img13.360buyimg.com/n7/jfs/t1/150978/1...,730.0,\n\t\t\t\t\t\t\t600+\n\t\t\t\t\t\t,https://item.jd.com/70265614548.html#comment,聚仁益智玩具专营店,https://mall.jd.com/index-751426.html?from=pc,NaN,广告,1,2021-01-27 18:34:00,https://search.jd.com/Search?keyword=%E4%B8%87...,600.0,其他
2,万代奥特曼,京东超市\t\n万代BANDAI奥特曼儿童玩具男孩动漫周边英雄超人赛罗奥特曼豪华版关节可动2...,100014813956,https://item.jd.com/100014813956.html,https://img11.360buyimg.com/n7/jfs/t1/150110/4...,129.0,\n\t\t\t\t\t\t\t3000+\n\t\t\t\t\t\t,https://item.jd.com/100014813956.html#comment,BANDAI万代奥特曼京东自营专区,https://mall.jd.com/index-1000353402.html?from=pc,自营/\r\n放心购/\r\n满199-20/,否,1,2021-01-27 18:34:00,https://search.jd.com/Search?keyword=%E4%B8%87...,3000.0,京东自营
3,万代奥特曼,万代 SHF成品可动模型手办 S.H.Figuarts奥特曼 Z泽塔赛罗捷德迪迦泰迦贝利亚...,10026088996068,https://item.jd.com/10026088996068.html,https://img13.360buyimg.com/n7/jfs/t1/167886/1...,279.0,\n\t\t\t\t\t\t\t100+\n\t\t\t\t\t\t,https://item.jd.com/10026088996068.html#comment,大白鲸动漫专营店,https://mall.jd.com/index-139432.html?from=pc,免邮/,否,1,2021-01-27 18:34:01,https://search.jd.com/Search?keyword=%E4%B8%87...,100.0,其他
4,万代奥特曼,京东超市\t\n万代BANDAI奥特曼儿童玩具男孩动漫周边英雄超人怪兽奈克赛斯奥特曼蓝色成年...,100003490309,https://item.jd.com/100003490309.html,https://img14.360buyimg.com/n7/jfs/t29335/183/...,79.0,\n\t\t\t\t\t\t\t1万+\n\t\t\t\t\t\t,https://item.jd.com/100003490309.html#comment,BANDAI万代奥特曼京东自营专区,https://mall.jd.com/index-1000353402.html?from=pc,自营/\r\n放心购/\r\n券199-100/,否,1,2021-01-27 18:34:01,https://search.jd.com/Search?keyword=%E4%B8%87...,10000.0,京东自营


In [8]:
toys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17706 entries, 0 to 17705
Data columns (total 17 columns):
搜索关键词    17706 non-null object
商品名称     17706 non-null object
商品SKU    17706 non-null int64
商品链接     17706 non-null object
封面图链接    17706 non-null object
价格       17706 non-null float64
评价人数     17706 non-null object
评论链接     17706 non-null object
商家店名     17702 non-null object
店铺链接     17702 non-null object
标签       14293 non-null object
是否广告     17706 non-null object
页码       17706 non-null int64
当前时间     17706 non-null object
页面网址     17706 non-null object
热度       17706 non-null float64
类型       17706 non-null object
dtypes: float64(2), int64(2), object(13)
memory usage: 2.3+ MB


# 数据探索

由于本次爬取的数据中并不包含单个商品实际购买数量，因此，我们用评论条数（只有购买的人可以进行评论）近似反映商品受欢迎的程度。以下我们来看看最受欢迎的玩具、最贵的玩具分别是什么。

In [9]:
#定义函数，按照数据行，抽取商品相关信息（包括图片），进行直观展示
def show_pic(row):
    print('-'*35,end='')
    print('%s : %s'%(row.类型, row.搜索关键词), end='')
    print('-'*35)    
    print(row.商品名称)
    print('价格：￥ %.2f' % row.价格)
    print('热度：%.2f' % row.热度)
    link=row.封面图链接
    img=imread(link)    
    plt.figure(figsize=(7,7))
    plt.imshow(img,aspect='equal')
    plt.axis("off")
    plt.show()

## 最受欢迎的产品

In [10]:
ix=toys.热度.idxmax()
show_pic(toys.iloc[ix])

-----------------------------------京东自营 : 乐高-----------------------------------
京东超市	
乐高(LEGO)积木 经典创意10698 大号积木盒 4-99岁 儿童玩具 男孩女孩 生日新年礼物 小颗粒
价格：￥ 419.00
热度：300000.00


<Figure size 504x504 with 1 Axes>

可以看到乐高的这款大号经典创意盒，评论条数达到了30万+，是所有玩具中最高的，推测原因有二：  
- 商品本身是乐高产品系列中价格适中，性价比较高的产品，销售量本身比较高  
- 来自京东自营店铺，店铺本身较高的信用度等也吸引了较多的消费者

## 最贵的产品

In [11]:
ix=toys.价格.idxmax()
show_pic(toys.iloc[ix])

-----------------------------------其他 : 乐高-----------------------------------
乐高 LEGO 10196 旋转木马 绝版 积木玩具
价格：￥ 39980.00
热度：0.00


<Figure size 504x504 with 1 Axes>

可以看到乐高的这款绝版旋转木马单价达到了近4万，是所有玩具中最高的，推测原因如下：  
- 绝版玩具，具有一定收藏价值  
- 玩具由其他类型商家销售，商品售价本身可能存在一定程度的虚高（从商品热度为0这一点也可以侧面印证）  
此外，我们可以尝试搜索其他商家，看是否同样的产品有更低的售价

In [12]:
mask=toys.商品名称.apply(lambda x: re.match('.*旋转木马.*',x)!=None)
ix_list=toys.index[mask]
for i in ix_list:
    show_pic(toys.iloc[i])

-----------------------------------其他 : 乐高-----------------------------------
乐高LEGO三合一创意creator系列恐龙汽车动物房屋积木三种拼法小颗粒拼装积木玩具 31095 游乐场旋转木马
价格：￥ 369.00
热度：3000.00


<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 乐高-----------------------------------
乐高 LEGO 10196 旋转木马 绝版 积木玩具
价格：￥ 39980.00
热度：0.00


<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 乐高-----------------------------------
乐高LEGO 创意百变三合一 儿童益智积木 拼插玩具 小颗粒 31095 游乐场旋转木马
价格：￥ 369.00
热度：100.00


...

<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 乐高-----------------------------------
【乐高认证店】乐高（LEGO）积木玩具 百变三合一系列 31095 游乐场旋转木马
价格：￥ 409.00
热度：2000.00


<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 乐高-----------------------------------
乐高LEGO经典创意五位数儿童玩具积木拼插 礼物 推荐款！ 10257旋转木马动力改装配件（不含主体）
价格：￥ 328.00
热度：200.00


<Figure size 504x504 with 1 Axes>

结果发现，这家店铺还真是独此一家，物以稀为贵，所以能卖出这样的天价。

# 数据分析及可视化

## 不同品类的价格分布

根据高达、奥特曼、乐高3个不同的玩具品类，分别对其产品价格分布进行可视化，并对可视化结果进行比较分析。由于价格分布较广，为了可视化的效果，需对价格取对数。

In [13]:
toy_dict={'高达':gundam,'奥特曼':outman,'乐高':lego}

In [14]:
for i in toy_dict:
    sns.distplot(np.log10(toy_dict[i].价格),label=i,hist=None)
plt.legend();

<Figure size 432x288 with 1 Axes>

分析上图可以发现：  
- 奥特曼类玩具主要分布于2个价格区间，分别是接近100元及100+元，其价格主要分布区间为三者最低。  
- 高达主要价格分布于300元左右(10^2.5)，价格位于其他两者中间  
- 乐高玩具价格分布与高达类似，但在1000元以上的区间中，仍有较高的分布比例

## 不同商家类型的价格分布

根据京东自营、品牌旗舰店、其他3个不同的商家类型，分别对其产品价格分布进行可视化，并对可视化结果进行比较分析。由于价格分布较广，为了可视化的效果，需对价格取对数。

In [15]:
cat_data=dict()
for i in toys.类型.unique():
    cat_data[i]=toys.loc[toys.类型==i]

In [16]:
for i in toys.类型.unique():
    sns.distplot(np.log10(cat_data[i].价格), label=i,hist=None)
plt.legend();

<Figure size 432x288 with 1 Axes>

从上图可以看出：  
- 三者价格分布区间差异不大，基本都位于100-1000元之间  
- 京东自营店铺价格可以看出较明显的3个峰值，都位于100-1000区间中  
- 品牌旗舰店在100-1000区间中有2个较明显的峰值，且右侧峰值较高，说明品牌旗舰店商品价格分布与京东自营类似，只是在1000元附近的商品更多一些  
- 其他店铺除了在100-100元区间中有一个分布峰值外，100元以下也有一个，提示其他店铺在100元以下的商品比例比京东自营店或旗舰店多

## 不同品类的热度分布

根据高达、奥特曼、乐高3个不同的玩具品类，分别对其产品热度分布进行可视化，并对可视化结果进行比较分析。由于热度分布较广，为了可视化的效果，需对价格取对数。（部分产品热度为0，为了防止数值错误，统一+1处理）

In [17]:
for i in toy_dict:
    sns.distplot(np.log10(toy_dict[i].热度+1),label=i,hist=None)
plt.legend();

<Figure size 432x288 with 1 Axes>

从上图可以大体上看出：  
- 3中品类都呈现3峰的分布，分别位于0热度、100热度、1000热度附近  
- 热度从高到底依次为乐高、奥特曼、高达

## 不同商家类型的热度分布

根据京东自营、品牌旗舰店、其他3个不同的商家类型，分别对其产品热度分布进行可视化，并对可视化结果进行比较分析。由于热度分布较广，为了可视化的效果，需对价格取对数。（部分产品热度为0，为了防止数值错误，统一+1处理）

In [18]:
for i in toys.类型.unique():
    sns.distplot(np.log10(cat_data[i].热度+1), label=i,hist=False)
plt.legend()
plt.ylim([0,1.5]);

<Figure size 432x288 with 1 Axes>

从上图中可以明显看出：  
- 大量品牌旗舰店的商品关注度为0  
- 其他类店铺关注度主要分布在3个峰值：0，100，1000，这与之前3类商品的关注分布类似  
- 京东自营店铺的大量商品关注度分布在100以上，甚至有部分商品关注度到了10万-100万区间，提示**京东自营的商品更加受消费者信赖**。

## 不同品类的购买倾向

通过对不同品类商品的购买数量在各个价格区间上的分布进行可视化，探索消费者在不同价格区间上，对各类商品的购买倾向。

In [19]:
#定义函数，根据商品数据，绘制商品价格分布（曲线），以及购买数量分布（直方图）
def compair_cv(df):
    test=[]
    for i in df.iterrows():
        a=[i[1].价格]
        test.extend(a*int(i[1].热度))
    sns.distplot(np.log10(test),kde=False,norm_hist=True,label='购买热度')
    sns.distplot(np.log10(df.价格),hist=False,label='价格分布')
    plt.legend()
    plt.xlim([0,5.0]);plt.ylim([0,4.5]) 

In [20]:
compair_cv(gundam)

<Figure size 432x288 with 1 Axes>

- 高达类产品  
相对于其价格区间，消费者更倾向于购买低价位产品（300元以下），甚至我们可以看到在100元以下，也有较高的热度，于此相对应，300元以上的产品虽然条数较多，但普遍热度较低。接下来，我们可以抽取并展示价格前5的高达产品信息进行验证：

In [21]:
toplist=gundam.sort_values(by=['价格'],ascending=False)[:5].index
for i in toplist:
    show_pic(gundam.loc[i])

-----------------------------------其他 : 万代高达-----------------------------------
万代（BANDAI） MS解体匠机 高达 模型手办 37cm （全款待定：时间待定）MS 沙扎比
价格：￥ 25000.00
热度：13.00


<Figure size 504x504 with 1 Axes>

-----------------------------------品牌旗舰店 : 万代高达-----------------------------------
现货速发万代METAL STRUCTURE 解体匠机 牛高达模型玩具 RX-93 1/60 日版 现货 随机  含运输箱
价格：￥ 21999.00
热度：0.00


<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 万代高达-----------------------------------
万代METAL STRUCTURE 解体匠机 牛高达模型玩具 RX-93 1/60圣诞玩具礼物 解体匠牛高达
价格：￥ 20000.00
热度：21.00


<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 万代高达-----------------------------------
生日礼物BANDAI  METAL BUILD MB 强袭自由高达 会场魂蓝再版初版光翼 强袭自由(魂限定再版)+光翼
价格：￥ 12500.00
热度：1.00


<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 万代高达-----------------------------------
高达模型AZA场景万代00rpg花海mg代工100情景60合金exia高达00 花海
价格：￥ 9100.00
热度：0.00


<Figure size 504x504 with 1 Axes>

可以看到与我们的判断一致，这些产品售价都较高（万元以上），但热度都在30以下。提示高达类高价产品的溢价普遍较高。

In [22]:
compair_cv(outman)

<Figure size 432x288 with 1 Axes>

- 奥特曼类产品  
可以看到奥特曼类产品的热度主要也都分布在其价格曲线峰值附近，300元以上的购买量较少，但与高达类产品不同，奥特曼类产品在高价区间购买量较低的主要原因可能是在300，甚至1000元以上本身产品数量就较少，因此热度较低也就不奇怪了。下面我们比较下1000元以上高达和奥特曼的产品数量：

In [23]:
count_outman=len(outman.loc[outman.价格>1000])
count_gundam=len(gundam.loc[gundam.价格>1000])
print('1000元以上奥特曼产品有%d款'% count_outman)
print('1000元以上高达产品有%d款'% count_gundam)

1000元以上奥特曼产品有244款
1000元以上高达产品有823款


In [24]:
compair_cv(lego)

<Figure size 432x288 with 1 Axes>

- 乐高类产品  
上图中可以看出，与高达类产品的高价位区间明显热度下降不同，乐高类产品的购买热度分布几乎与其价格曲线一致，因此我们可以认为，各个价格段的乐高产品都得到了消费者一定程度上的认可。对此，我们可以比较1000元以上区间内高达产品及乐高产品的热度来进行验证：

In [25]:
#1000元以上高达类产品热度排名前3
toplist=gundam.loc[gundam.价格>1000,:].sort_values(by=['热度'],ascending=False)[:3].index
for i in toplist:
    show_pic(gundam.loc[i])

-----------------------------------其他 : 万代高达-----------------------------------
万代（BANDAI） RG 高达2 拼装模型玩具 14cm RG 独角兽3号机电镀
价格：￥ 1199.00
热度：3000.00


<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 万代高达-----------------------------------
万代（BANDAI） MG 高达2 拼装模型玩具 18cm MG 深度强袭
价格：￥ 1290.00
热度：2000.00


<Figure size 504x504 with 1 Axes>

-----------------------------------其他 : 万代高达-----------------------------------
万代（BANDAI） PG 高达 拼装模型玩具 30cm PG 00R强化模组
价格：￥ 1490.00
热度：1000.00


<Figure size 504x504 with 1 Axes>

In [26]:
#1000元以上乐高类产品热度排名前3
toplist=lego.loc[lego.价格>1000,:].sort_values(by=['热度'],ascending=False)[:3].index
for i in toplist:
    show_pic(lego.loc[i])

-----------------------------------京东自营 : 乐高-----------------------------------
京东超市	
乐高(LEGO)积木 得宝10933 忙碌的建筑基地2岁+大颗粒早教儿童玩具  吊车挖掘车 男孩女孩 生日礼物6月上新
价格：￥ 1059.00
热度：200000.00


<Figure size 504x504 with 1 Axes>

-----------------------------------京东自营 : 乐高-----------------------------------
京东超市	
乐高(LEGO)积木 机械组Technic保时捷 911 RSR赛车10岁+ 42096 生日新年礼物 儿童玩具 男孩女孩 粉丝收藏
价格：￥ 1199.00
热度：100000.00


<Figure size 504x504 with 1 Axes>

-----------------------------------京东自营 : 乐高-----------------------------------
京东超市	
乐高(LEGO)积木 IDEAS系列92176 美国宇航局阿波罗土星五号14岁+生日新年礼物 儿童玩具 男孩女孩11月上新
价格：￥ 1099.00
热度：100000.00


<Figure size 504x504 with 1 Axes>

从以上结果可以看出，两者热度差异巨大，高达类产品热度为1000-3000，乐高类为10万-20万，这验证了我们之前的结论。

## 不同商家类型的购买倾向

In [27]:
#其他
compair_cv(cat_data['其他'])

<Figure size 432x288 with 1 Axes>

In [28]:
#品牌旗舰店
compair_cv(cat_data['品牌旗舰店'])

<Figure size 432x288 with 1 Axes>

In [29]:
#京东自营
compair_cv(cat_data['京东自营'])

<Figure size 432x288 with 1 Axes>

比较以上3图，可以发现：  
- 对于其他类店铺，购买热度分布相对价格曲线向左偏移，提示对于其他类店铺，消费者更倾向于购买低价位产品  
- 对于品牌旗舰店，购买热度分布相对价格曲线向右偏移，提示对于品牌旗舰店，消费者更倾向于购买高价位产品  
- 对于京东自营店铺，购买热度分布与价格曲线基本一致，提示对于京东自营店铺，消费者没有比较明显的购买倾向

# 结论

综上，我们已经可以回答开头提出的几个问题了

## 最受欢迎的产品是什么？

In [30]:
ix=toys.热度.idxmax()
show_pic(toys.iloc[ix])

-----------------------------------京东自营 : 乐高-----------------------------------
京东超市	
乐高(LEGO)积木 经典创意10698 大号积木盒 4-99岁 儿童玩具 男孩女孩 生日新年礼物 小颗粒
价格：￥ 419.00
热度：300000.00


<Figure size 504x504 with 1 Axes>

## 什么品牌的产品单价最贵？

In [31]:
ix=toys.价格.idxmax()
show_pic(toys.iloc[ix])

-----------------------------------其他 : 乐高-----------------------------------
乐高 LEGO 10196 旋转木马 绝版 积木玩具
价格：￥ 39980.00
热度：0.00


<Figure size 504x504 with 1 Axes>

## 单价高的产品是否存在溢价？

通过前面的分析，我们可以得出结论：  
- 对于奥特曼类产品，高价区间产品品类本身较少  
- 对于高达类产品，高价区间有较多产品并没有获得很高的购买热度，这可能是高溢价导致的  
- 对于乐高类产品，与高达不同，虽然高价区间有很多产品，但销量相对高达类都更加客观，证明消费者比较认可高价位的乐高产品

## 能否给过年送礼一些建议？

- 根据品类的丰富程度，如果选择低价位（100元以下）的礼物，奥特曼类产品选择较多，中高价位(100-1000元），乐高和高达类产品可选都较多。  
- 从产品热度来看，乐高类产品明显更受欢迎，如果不知道选什么的话，选乐高还是比较保险的  
- 从商家类型来看，大家对京东自营店铺更加认可，建议选择

## 消费者更喜欢在旗舰店购买产品吗？

从不同商家类型的产品热度分布比较来看，京东自营店铺的热度明显高于品牌旗舰店和其他店铺类型。